## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()  #keras自带的MNIST数据集加载方法

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 52s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
#OneHotEncoder()也就是说，一个属性如果有N个可取值，它就可以扩充为N个属性，每个样本的这N个属性中，只能有一个为1，表示该样本的该属性属于这个类别，其余扩展属性都为0。
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

D:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:

classifier=Sequential()

#卷積組合
#classifier.add(Convolution2D('自己設計參數'))#32,3,3,input_shape=(32,32,3),activation='relu''
#classifier.add(BatchNormalization())

classifier.add(Convolution2D(32,kernel_size=(3, 3), padding='same',input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())


'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
#classifier.add(Convolution2D('自己設計參數'))
#classifier.add(BatchNormalization())
classifier.add(Convolution2D(64,kernel_size=(3, 3), padding='same',activation='relu'))
classifier.add(BatchNormalization())


classifier.add(MaxPooling2D(pool_size=(2,2)))#放在這
#flatten
classifier.add(Flatten())

#FC
#classifier.add(Dense('自己設計FC層參數')) #output_dim=100,activation=relu
classifier.add(Dense(output_dim=128,activation='relu')) #output_dim=100,activation=relu

#輸出
#classifier.add(Dense(output_dim=10,activation='輸出函數應該用什麼？'))
#Softmax函數能將輸出值總合統整成1，轉換成機率的型態，通常⽤用於多類的分類器輸出。
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy  
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
'''
#optimizer: 优化器 
#loss:损失函数,可以用自带的,也可以自定义.如果模型有多个输出,可以传入一个字典或者损失列表,模型降会把这些损失加在一起
metrics: 评价函数,与损失函数类似,只不过评价函数的结果不会用于训练过程中
,可以传递已有的评价函数名称,或者传递一个自定义的theano/tensorflow函数来使用
,自带的评价函数有:binary_accuracy(y_true,y_pred)
, categorical_accuracy(y_true,y_pred)
,sparse_categorical_accuracy(y_true,y_pred)
, top_k_categorical_accuracy(y_true,y_pred,k=5).
自定义评价函数应该在编译的时候compile传递进去,该函数需要以(y_true,y_pred)作为输入参数,并返回一个张量作为输出结果.
loss_weights:可选项,是一个list或字典,指定不同的损失的系数
'''
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

D:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
D:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 157s 3ms/step - loss: 1.4703 - accuracy: 0.4860
Epoch 2/100
50000/50000 [==============================] - 144s 3ms/step - loss: 0.9999 - accuracy: 0.6429
Epoch 3/100
50000/50000 [==============================] - 161s 3ms/step - loss: 0.8246 - accuracy: 0.7098
Epoch 4/100
50000/50000 [==============================] - 149s 3ms/step - loss: 0.6991 - accuracy: 0.7573
Epoch 5/100
50000/50000 [==============================] - 150s 3ms/step - loss: 0.5674 - accuracy: 0.8029
Epoch 6/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.4466 - accuracy: 0.8469
Epoch 7/100
50000/50000 [==============================] - 168s 3ms/step - loss: 0.3422 - accuracy: 0.8831
Epoch 8/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.2576 - accuracy: 0.9114
Epoch 9/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.1993 - accuracy: 0.9323
Epoch 10/100
50000/50000 [===========

50000/50000 [==============================] - 154s 3ms/step - loss: 0.0187 - accuracy: 0.9943
Epoch 77/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0172 - accuracy: 0.9946
Epoch 78/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0155 - accuracy: 0.9950
Epoch 79/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0157 - accuracy: 0.9952
Epoch 80/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0198 - accuracy: 0.9943
Epoch 81/100
50000/50000 [==============================] - 155s 3ms/step - loss: 0.0213 - accuracy: 0.9939
Epoch 82/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0190 - accuracy: 0.9940
Epoch 83/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0141 - accuracy: 0.9955
Epoch 84/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0216 - accuracy: 0.9936
Epoch 85/100
50000/50000 [===============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.0284973e-15, 1.3892687e-22, 3.5717175e-26, 1.0000000e+00,
        2.2147266e-09, 1.3937302e-26, 3.9160554e-24, 7.0002900e-26,
        4.9680798e-18, 1.1462182e-28]], dtype=float32)